In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
WAVELENGTHS = np.arange(380, 781, 1) # 파장 범위 및 간격 (nm), 예시로 5nm 간격
NUM_WAVELENGTHS = len(WAVELENGTHS)
DRIVE_PATH = "C:/Users/PC/Desktop/Deep/Materials_380"
DATA_DIR = DRIVE_PATH
# 재료별 n, k 데이터 로드


material_files = [
    #('HfO2', 'HfO2_380.csv'), # 사용자 코드에서 가져온 변수명 기반 추정
    ('LiF',  'LiF_380.csv'),
    ('SiO2', 'SiO2_380.csv'),
    ('WO3',  'WO3_380.csv'),
    #('ZnS',  'ZnS_380.csv'),
    #('Al2O3',  'Al2O3_380.csv'),
    #('MgF2',  'MgF2_380.csv'),
    #('TiO2',  'TiO2_380.csv'),
    ('Ag' , 'Ag_380.csv')
]
material_data = {}
material_names = [info[0] for info in material_files]
print(material_names)

material_names = sorted(set(info[0] for info in material_files))  # 순서 고정
material_to_index = {name: i for i, name in enumerate(material_names)}
index_to_material = {i: name for name, i in material_to_index.items()}
print(index_to_material)


# --- 절연체 파일 처리 ---
for material_name, filename in material_files:
    file_path = os.path.join(DATA_DIR, filename)
    print(f"처리 중: {filename} (materials: {material_name})")
    data = pd.read_csv(file_path, header=None)

    num_samples_in_file = data.shape[0]
    if num_samples_in_file == 0:
        print(f"  경고: 파일이 비어있습니다.")
        continue
    print(f"  로드된 샘플 수: {num_samples_in_file}")

    # 파장, n, k 분리
    wavelengths = data.iloc[:,
        0].values  # 첫 번째 열: 파장
    n = data.iloc[:, 1].values            # 두 번째 열: n
    k = data.iloc[:, 2].values            # 세 번째 열: k

    # 복소 굴절률 생성
    n_complex = n + 1j * k

    # insulator_data에 저장
    material_data[material_name] = (wavelengths, n_complex)
# --- 공기 굴절률 처리 ---
material_data['Air'] = np.ones(NUM_WAVELENGTHS, dtype=np.complex128)


['LiF', 'SiO2', 'WO3', 'Ag']
{0: 'Ag', 1: 'LiF', 2: 'SiO2', 3: 'WO3'}
처리 중: LiF_380.csv (materials: LiF)
  로드된 샘플 수: 401
처리 중: SiO2_380.csv (materials: SiO2)
  로드된 샘플 수: 401
처리 중: WO3_380.csv (materials: WO3)
  로드된 샘플 수: 401
처리 중: Ag_380.csv (materials: Ag)
  로드된 샘플 수: 401


In [ ]:
# ────────────────────────────────────────────────────────────────────────────
# (A) 환경 및 TMM 파라미터 준비
# ────────────────────────────────────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# (1) 파장 정의 (380~780 nm, 5 nm 간격 → 81개)
WAVELENGTHS = np.arange(380, 781, 1)            # [380, 385, ..., 780]
NUM_WAVELENGTHS = len(WAVELENGTHS)             # 81
λ_tensor_global = torch.tensor(WAVELENGTHS, dtype=torch.float64).to(device) * 1e-9  # [m 단위]

material_sequence = ['Ag', 'WO3', 'Ag']

#    각 층(material_sequence)의 복소 n(λ) 배열을 numpy로부터 추출
n_list_np_arrays = [material_data[mat][1] for mat in material_sequence]  # 길이 3 리스트


#    numpy → torch로 변환 & device로 이동 (dtype=torch.cdouble)
n_list_torch = torch.stack([
    torch.from_numpy(arr).to(torch.complex64) for arr in n_list_np_arrays
], dim=0).to(device)

print(n_list_torch.shape)

#    입사·출사 매질 굴절률 (예: 입사=SiO2, 출사=Air)
n_i_np = material_data['SiO2'][1]   # numpy complex
n_s_np = material_data['Air']       # numpy complex

n_i_torch = torch.from_numpy(n_i_np).to(torch.complex64).to(device)
n_s_torch = torch.from_numpy(n_s_np).to(torch.complex64).to(device)

# (4) 두께 그리드 (nm 단위)
d1_list_nm = np.arange(5, 41, 5)       # [5, 10, 15, 20, 25, 30]
d2_list_nm = np.arange(150, 801, 5)  # [150, 160, …, 1000]
d3_list_nm = np.arange(5, 41, 5)        # [5, 10, 15, 20, 25, 30]
print(len(d1_list_nm)*len(d2_list_nm)*len(d3_list_nm))
# (5) 샘플 저장용 리스트
all_d_tilde = []    # [(tilde1, tilde2, tilde3), …]
all_T_target = []   # [array([T(λ), …, T(λ)]), …]

Using device: cuda
torch.Size([3, 401])
8384


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

# ────────────────────────────────────────────────────────────────────────────
# (A) 환경 및 TMM 파라미터 준비
# ────────────────────────────────────────────────────────────────────────────

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# (1) 파장 정의 (380~780 nm, 1 nm 간격 → 401개)
WAVELENGTHS = np.arange(380, 781, 1)            # [380, 381, ..., 780] (nm)
NUM_WAVELENGTHS = len(WAVELENGTHS)              # 401
# 반드시 “nm → m 단위 torch.Tensor” 형태로 넘겨야 함
λ_tensor_global = (
    torch.tensor(WAVELENGTHS, dtype=torch.float64)
    .to(device)
    * 1e-9
)  # [m 단위], shape: (401,)

# (2) material_data에서 복소 굴절률 로드
#    material_sequence: 3층 구조 예시
material_sequence = ['Ag', 'WO3', 'Ag']
n_list_np_arrays = [material_data[mat][1] for mat in material_sequence]  # [ (401,), (401,), (401,) ]

#    numpy → torch로 변환 & device로 이동 (dtype=torch.complex128)
n_list_torch = torch.stack([
    torch.from_numpy(arr).to(torch.complex128) for arr in n_list_np_arrays
], dim=0).to(device)  # shape: (3, 401), dtype: complex128

#    입사·출사 매질 굴절률 (예: 입사=SiO2, 출사=Air)
n_i_np = material_data['SiO2'][1]   # numpy complex128 shape: (401,)
n_s_np = material_data['Air']       # numpy complex128 shape: (401,)

n_i_torch = torch.from_numpy(n_i_np).to(torch.complex128).to(device)  # shape: (401,)
n_s_torch = torch.from_numpy(n_s_np).to(torch.complex128).to(device)  # shape: (401,)

Using device: cuda


In [ ]:
# ────────────────────────────────────────────────────────────────────────────
# (A-1) TMMNetwork 정의 (클래스 이름, dtype 통일, calculate() 추가)
# ────────────────────────────────────────────────────────────────────────────
class TMMNetwork(nn.Module):
    def __init__(self, n_list_input, n_i_input, n_s_input, wavelengths_m_tensor):
        super().__init__()
        dtype_complex = torch.complex128
        dtype_float = torch.float64

        # 기준길이 L0 (이제 tilde는 안 쓰므로 없어도 무방하지만 유지 가능)
        n_list_real = n_list_input.real
        n_list_imag = -torch.abs(n_list_input.imag)
        self.register_buffer('n_list', torch.complex(n_list_real, n_list_imag))
        self.register_buffer('n_i', n_i_input.to(dtype_complex))
        self.register_buffer('n_s', n_s_input.to(dtype_complex))

        # 파수 k0
        k0 = 2 * torch.pi / torch.clamp(wavelengths_m_tensor, min=1e-20)
        self.register_buffer('k0', k0)  # 이제 k0_tilde 아님

        self.num_layers = self.n_list.shape[0]
        self.num_wavelengths = wavelengths_m_tensor.shape[0]

    def forward(self, thicknesses_nm):
        device = thicknesses_nm.device
        dtype_complex = torch.complex128
        imag_unit = torch.tensor(1j, dtype=dtype_complex, device=device)

        if thicknesses_nm.ndim == 1:
            thicknesses_nm = thicknesses_nm.unsqueeze(0)  # (1, 3)
        B = thicknesses_nm.shape[0]  # batch size

        # Convert to meters
        thicknesses_m = thicknesses_nm * 1e-9  # (B, 3)

        # (B, 3, 401): broadcasting layer × wavelength
        delta = thicknesses_m[:, :, None] * self.k0[None, None, :] * self.n_list[None, :, :]

        cosδ = torch.cos(delta)
        sinδ = torch.sin(delta)

        # 초기 M_total: (B, 401, 2, 2)
        M_total = torch.eye(2, dtype=dtype_complex, device=device).repeat(B, self.num_wavelengths, 1, 1)

        for j in range(self.num_layers):
            Yj = self.n_list[j] * 2.654e-3  # (401,)

            sin_j = sinδ[:, j, :]
            cos_j = cosδ[:, j, :]

            m11 = cos_j
            m12 = imag_unit * sin_j / Yj[None, :]
            m21 = imag_unit * Yj[None, :] * sin_j
            m22 = cos_j

            Mj = torch.stack([
                torch.stack([m11, m12], dim=-1),
                torch.stack([m21, m22], dim=-1)
            ], dim=-2)  # shape: (B, 401, 2, 2)

            M_total = torch.matmul(M_total, Mj)

        Y_in = self.n_i * 2.654e-3
        Y_out = self.n_s * 2.654e-3

        B_ = M_total[:, :, 0, 0] + M_total[:, :, 0, 1] * Y_out
        C_ = M_total[:, :, 1, 0] + M_total[:, :, 1, 1] * Y_out
        denom = Y_in * B_ + C_
        t1 = (2 * Y_in) / denom  # (B, 401)

        T = (torch.real(self.n_s) / torch.real(self.n_i)) * torch.abs(t1) ** 2
        return T.to(torch.float64)  # (B, 401)
# ────────────────────────────────────────────────────────────────────────────
# (A-2) 단일 시험용 스펙트럼 계산 (디버깅 / 시각화)
# ────────────────────────────────────────────────────────────────────────────
model = TMMNetwork(n_list_torch, n_i_torch, n_s_torch, λ_tensor_global).to(device)

In [ ]:
import os
import random
import numpy as np
import torch

# 시드 고정 (재현성 확보)

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)  # 시드 고정 실행

In [ ]:
# @title
from torch.utils.data import random_split
all_d = []
all_T_target = []

print("Generating (normalized_d → T_target) pairs ...")
for d1 in d1_list_nm:
    for d2 in d2_list_nm:
        for d3 in d3_list_nm:
            # (a) [d1, d2, d3]
            d_nm_vec = torch.tensor([[d1, d2, d3]], dtype=torch.float64)  # shape: (1, 3)
            all_d.append(d_nm_vec.cpu().numpy())
            # (b) TMM forward
            T_spec = model(d_nm_vec.to(device))  # forward(nm 단위 두께)
            all_T_target.append(T_spec.detach().cpu().numpy())
# NumPy 배열로 변환
all_d = np.array(all_d, dtype=np.float64)   # shape: (3096, 3)
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ두께표준화ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
THICKNESS_MIN = torch.tensor(np.min(all_d, axis=0), dtype=torch.float64, device=device)
THICKNESS_MAX = torch.tensor(np.max(all_d, axis=0), dtype=torch.float64, device=device)
def standardize_thickness(d_nm_array):
    d_nm_array = torch.tensor(d_nm_array, dtype=torch.float64, device=THICKNESS_MIN.device) if not isinstance(d_nm_array, torch.Tensor) else d_nm_array.to(dtype=torch.float64, device=THICKNESS_MIN.device)
    return (d_nm_array - THICKNESS_MIN) / (THICKNESS_MAX - THICKNESS_MIN)

def destandardize_thickness(d_norm_array):
    d_norm_array = d_norm_array.clone().to(dtype=torch.float64, device=THICKNESS_MIN.device)
    return d_norm_array * (THICKNESS_MAX - THICKNESS_MIN) + THICKNESS_MIN

all_d_norm = standardize_thickness(all_d)  # shape: (3096, 3)
d_nm_check = destandardize_thickness(all_d_norm)


#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ스펙트럼표준화ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
all_T_target = np.array(all_T_target, dtype=np.float64) # shape: (3096, 401)
print("Total samples:", all_d.shape[0])  # 3096

SPECTRUM_MIN = torch.tensor(np.min(all_T_target, axis=0), dtype=torch.float64).to(device)
SPECTRUM_MAX = torch.tensor(np.max(all_T_target, axis=0), dtype=torch.float64).to(device)
def standardize_spectrum(T_array):
    T_tensor = torch.tensor(T_array, dtype=torch.float64).to(device)
    return (T_tensor - SPECTRUM_MIN) / (SPECTRUM_MAX - SPECTRUM_MIN)

def destandardize_spectrum(T_std_array):
    return T_std_array * (SPECTRUM_MAX - SPECTRUM_MIN) + SPECTRUM_MIN

# destandardize_spectrum의 결과(GPU 텐서)를 .cpu().numpy()로 변환하여 비교합니다.
all_T_target_std = standardize_spectrum(all_T_target)
restored_T = destandardize_spectrum(all_T_target_std)
assert np.allclose(restored_T.cpu().numpy(), all_T_target, atol=1e-5)

# ────────────────────────────────────────────────────────────────────────────
# (B) Dataset / DataLoader 구축
# ────────────────────────────────────────────────────────────────────────────
class BiTMMNormalizedDataset(Dataset):
    def __init__(self, d_norm_array, T_array):
        self.d_norm = d_norm_array.clone().to(dtype=torch.float64)
        self.T_spec = T_array.clone().detach().to(dtype=torch.float64) if isinstance(T_array, torch.Tensor) else torch.tensor(T_array, dtype=torch.float64)

    def __len__(self):
        return self.d_norm.shape[0]

    def __getitem__(self, idx):
        return {
            'd_norm': self.d_norm[idx],       # shape: (3,)1
            'T_target': self.T_spec[idx]      # shape: (401,)
        }

batch_size = 32


# 시드 고정
seed = 42
g = torch.Generator().manual_seed(seed)

# Dataset 생성
dataset = BiTMMNormalizedDataset(all_d_norm, all_T_target_std)

# 전체 길이 및 split 비율
total_size = len(dataset)
train_size = int(0.9 * total_size)
val_size   = int(0.05 * total_size)
test_size  = total_size - train_size - val_size  # 나머지

print(f"Total: {total_size}, Train: {train_size}, Val: {val_size}, Test: {test_size}")

# Dataset 분할
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size], generator=g  # g로 통일
)

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=0, generator=g)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=0)


Generating (normalized_d → T_target) pairs ...
Total samples: 8384
Total: 8384, Train: 7545, Val: 419, Test: 420


In [ ]:
# @title
from torch.utils.data import random_split
all_d = []
all_T_target = []

print("Generating (normalized_d → T_target) pairs ...")
for d1 in d1_list_nm:
    for d2 in d2_list_nm:
        for d3 in d3_list_nm:
            # (a) [d1, d2, d3]
            d_nm_vec = torch.tensor([[d1, d2, d3]], dtype=torch.float64)  # shape: (1, 3)
            all_d.append(d_nm_vec.cpu().numpy())
            # (b) TMM forward
            T_spec = model(d_nm_vec.to(device))  # forward(nm 단위 두께)
            all_T_target.append(T_spec.detach().cpu().numpy())
# NumPy 배열로 변환
all_d = np.array(all_d, dtype=np.float64)   # shape: (3096, 3)
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ두께표준화ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
THICKNESS_MEAN = torch.tensor(np.mean(all_d, axis=0), dtype=torch.float64, device=device)
THICKNESS_STD  = torch.tensor(np.std(all_d, axis=0),  dtype=torch.float64, device=device)
def standardize_thickness(d_nm_array):
    if not isinstance(d_nm_array, torch.Tensor):
        d_nm_array = torch.tensor(d_nm_array, dtype=torch.float64, device=THICKNESS_MEAN.device)
    else:
        d_nm_array = d_nm_array.to(dtype=torch.float64, device=THICKNESS_MEAN.device)
    return (d_nm_array - THICKNESS_MEAN) / THICKNESS_STD

def destandardize_thickness(d_norm_array):
    d_norm_array = d_norm_array.clone().to(dtype=torch.float64, device=THICKNESS_MEAN.device)
    return d_norm_array * THICKNESS_STD + THICKNESS_MEAN

all_d_norm = standardize_thickness(all_d)  # shape: (3096, 3)
d_nm_check = destandardize_thickness(all_d_norm)


#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ스펙트럼표준화ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
all_T_target = np.array(all_T_target, dtype=np.float64) # shape: (3096, 401)
print("Total samples:", all_d.shape[0])  # 3096

SPECTRUM_MEAN = torch.tensor(np.mean(all_T_target, axis=0), dtype=torch.float64).to(device)
SPECTRUM_STD = torch.tensor(np.std(all_T_target, axis=0), dtype=torch.float64).to(device)
def standardize_spectrum(T_array):
    # 1. 입력받은 NumPy 배열(T_array)을 PyTorch 텐서로 변환하고 GPU로 보냅니다.
    T_tensor = torch.tensor(T_array, dtype=torch.float64).to(device)

    # 2. 이제 GPU 텐서끼리의 연산을 수행합니다.
    return (T_tensor - SPECTRUM_MEAN) / SPECTRUM_STD
def destandardize_spectrum(T_std_array):
    return T_std_array * SPECTRUM_STD + SPECTRUM_MEAN

# destandardize_spectrum의 결과(GPU 텐서)를 .cpu().numpy()로 변환하여 비교합니다.
all_T_target_std = standardize_spectrum(all_T_target)
restored_T = destandardize_spectrum(all_T_target_std)
assert np.allclose(restored_T.cpu().numpy(), all_T_target, atol=1e-5)

# ────────────────────────────────────────────────────────────────────────────
# (B) Dataset / DataLoader 구축
# ────────────────────────────────────────────────────────────────────────────
class BiTMMNormalizedDataset(Dataset):
    def __init__(self, d_norm_array, T_array):
        self.d_norm = d_norm_array.clone().to(dtype=torch.float64)
        self.T_spec = T_array.clone().detach().to(dtype=torch.float64) if isinstance(T_array, torch.Tensor) else torch.tensor(T_array, dtype=torch.float64)

    def __len__(self):
        return self.d_norm.shape[0]

    def __getitem__(self, idx):
        return {
            'd_norm': self.d_norm[idx],       # shape: (3,)1
            'T_target': self.T_spec[idx]      # shape: (401,)
        }


# 시드 고정
seed = 42
g = torch.Generator().manual_seed(seed)

# Dataset 생성
dataset = BiTMMNormalizedDataset(all_d_norm, all_T_target_std)

# 전체 길이 및 split 비율
total_size = len(dataset)
train_size = int(0.9 * total_size)
val_size   = int(0.05 * total_size)
test_size  = total_size - train_size - val_size  # 나머지

print(f"Total: {total_size}, Train: {train_size}, Val: {val_size}, Test: {test_size}")

# Dataset 분할
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size], generator=g  # g로 통일
)

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=0, generator=g)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=0)


Generating (normalized_d → T_target) pairs ...
Total samples: 7399
Total: 7399, Train: 6659, Val: 369, Test: 371


# **Forward Training **

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

# ──────────────────────────────────────────────────────
# 모델 정의
# ──────────────────────────────────────────────────────

batch_size = 128

N1 = 1190
N2 = 824
N3 = 920

class ForwardNet(nn.Module):
    def __init__(self, input_dim=3, output_dim=401):
        super().__init__()
        self.net = nn.Sequential(
    nn.Linear(3, N1),
    nn.ReLU(),
    nn.Linear(N1, N2),
    nn.ReLU(),
    nn.Linear(N2, N3),
    nn.ReLU(),
    nn.Linear(N3, 401)
)

    def forward(self, x):
        return self.net(x)

# 모델 및 옵티마이저 초기화
forward_net = ForwardNet(input_dim=3, output_dim=401).to(device).to(torch.float64)
optimizer = torch.optim.Adam(forward_net.parameters(), lr=1e-3)

# ──────────────────────────────────────────────────────
# 학습 파라미터 설정
# ──────────────────────────────────────────────────────
num_epochs =600
output_dir = "C:/Users/PC/Desktop/Deep/"
os.makedirs(output_dir, exist_ok=True)

train_loss_history = []
val_loss_history = []
# ──────────────────────────────────────────────────────
# 학습 루프
# ──────────────────────────────────────────────────────
for epoch in range(num_epochs):
    forward_net.train()
    total_loss = 0.0

    for batch in train_loader:
        d_norm = batch['d_norm'].to(device)        # [B, 3]
        T_target = batch['T_target'].to(device)    # [B, 401]

        optimizer.zero_grad()
        T_pred = forward_net(d_norm)
        loss = F.mse_loss(T_pred, T_target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    train_loss_history.append(avg_train_loss)
    print(f"[Train] Epoch {epoch+1}, Avg Loss = {avg_train_loss:.6f}")

    # ───────────────────────────────────────────
    # Validation
    # ───────────────────────────────────────────
    forward_net.eval()
    val_loss_total = 0.0
    with torch.no_grad():
        for batch in val_loader:
            d_norm = batch['d_norm'].to(device)
            T_target = batch['T_target'].to(device)

            T_pred = forward_net(d_norm)
            loss = F.mse_loss(T_pred, T_target)
            val_loss_total += loss.item()

    avg_val_loss = val_loss_total / len(val_loader)
    val_loss_history.append(avg_val_loss)
    print(f"[Val]   Epoch [{epoch+1}/{num_epochs}] done. Avg Loss: {avg_val_loss:.6f}")

[Train] Epoch 1, Avg Loss = 0.025017
[Val]   Epoch [1/600] done. Avg Loss: 0.022380
[Train] Epoch 2, Avg Loss = 0.022529
[Val]   Epoch [2/600] done. Avg Loss: 0.021840
[Train] Epoch 3, Avg Loss = 0.021614
[Val]   Epoch [3/600] done. Avg Loss: 0.020993
[Train] Epoch 4, Avg Loss = 0.020435
[Val]   Epoch [4/600] done. Avg Loss: 0.018758
[Train] Epoch 5, Avg Loss = 0.018667
[Val]   Epoch [5/600] done. Avg Loss: 0.017885
[Train] Epoch 6, Avg Loss = 0.016694
[Val]   Epoch [6/600] done. Avg Loss: 0.015015
[Train] Epoch 7, Avg Loss = 0.014739
[Val]   Epoch [7/600] done. Avg Loss: 0.013633
[Train] Epoch 8, Avg Loss = 0.013267
[Val]   Epoch [8/600] done. Avg Loss: 0.012304
[Train] Epoch 9, Avg Loss = 0.012018
[Val]   Epoch [9/600] done. Avg Loss: 0.011191
[Train] Epoch 10, Avg Loss = 0.010457
[Val]   Epoch [10/600] done. Avg Loss: 0.010122
[Train] Epoch 11, Avg Loss = 0.009094
[Val]   Epoch [11/600] done. Avg Loss: 0.007966
[Train] Epoch 12, Avg Loss = 0.007542
[Val]   Epoch [12/600] done. Avg L

In [ ]:
# ────────────────────────────────────────────────────────────────────────────
# (C) InverseNet 정의 (MLP)
# ────────────────────────────────────────────────────────────────────────────
t_path = '/content/drive/MyDrive/Colab Notebooks/Bi-directional20250402'
os.makedirs(t_path, exist_ok=True)
num_layers = len(material_sequence)  # 3
class InverseNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 1198),
            nn.ReLU(),
            nn.Linear(1198, 1125),
            nn.ReLU(),
            nn.Linear(1125, 853),
            nn.ReLU(),
            nn.Linear(853, 1411),
            nn.ReLU(),
            nn.Linear(1411, 1320),
            nn.ReLU(),
            nn.Linear(1320, 1958),
            nn.ReLU(),
            nn.Linear(1958, output_dim)
        )

    def forward(self, x):
        # x must be shape (B, input_dim)
        return self.net(x)

# Re-initialize the model and optimizer after changing the class definition
inverse_net = InverseNet(input_dim=NUM_WAVELENGTHS, output_dim=num_layers).to(device)
inverse_net = inverse_net.to(torch.float64)
optimizer = optim.Adam(inverse_net.parameters(), lr=0.0016277391324458217)

# ────────────────────────────────────────────────────────────────────────────
# (D) 학습 루프: bi-TMM 방식
# ────────────────────────────────────────────────────────────────────────────
num_epochs = 600
save_interval = 600
output_dir = "C:/Users/PC/Desktop/Deep/"
os.makedirs(output_dir, exist_ok=True)
alpha = 1.0616785666328346
forward_net.eval()
for epoch in range(num_epochs):
    inverse_net.train()
    total_loss = 0.0
    for batch_idx, batch in enumerate(train_loader):
        T_target = batch['T_target'].to(device)      # [B, 401]
        d_norm_true = batch['d_norm'].to(device)     # [B, 3]

        optimizer.zero_grad()
        d_norm_pred = inverse_net(T_target.to(torch.float64)).squeeze(1)

        if batch_idx == 0:
            print(f"\n[DEBUG] Epoch {epoch+1}, Batch {batch_idx+1}")
        d_nm_pred = destandardize_thickness(d_norm_pred)
        d_nm_true = destandardize_thickness(d_norm_true)

        T_pred_batch = model(d_nm_pred)
        squeezed_T_target = T_target.squeeze(1)
        T_target_destd = destandardize_spectrum(squeezed_T_target)

        loss_spectrum = F.mse_loss(T_pred_batch, T_target_destd)
        loss_thickness = F.mse_loss(d_norm_pred, d_norm_true.squeeze(1))

        if epoch < 18:
            loss = loss_spectrum + alpha * loss_thickness
        else:
            loss = loss_spectrum

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"[Train] Epoch [{epoch+1}/{num_epochs}] done. Avg Loss: {avg_train_loss:.6f}")

    # ───────────────────────────────────────────
    # Validation
    # ───────────────────────────────────────────
    inverse_net.eval()
    val_loss_total = 0.0
    with torch.no_grad():
        for val_batch in val_loader:
            T_val = val_batch['T_target'].to(device)
            d_norm_val = val_batch['d_norm'].to(device)

            d_pred_val = inverse_net(T_val.to(torch.float64)).squeeze(1)
            d_nm_val_pred = destandardize_thickness(d_pred_val)
            T_val_pred = model(d_nm_val_pred)

            T_val_destd = destandardize_spectrum(T_val.squeeze(1))

            loss_val_spectrum = F.mse_loss(T_val_pred, T_val_destd)
            loss_val_thickness = F.mse_loss(d_pred_val, d_norm_val.squeeze(1))

            if epoch < 18:
                val_loss = loss_val_spectrum + alpha * loss_val_thickness
            else:
                val_loss = loss_val_spectrum

            val_loss_total += val_loss.item()

    avg_val_loss = val_loss_total / len(val_loader)
    print(f"[Val]   Epoch [{epoch+1}/{num_epochs}] done. Avg Loss: {avg_val_loss:.6f}")
# 저장할 경로와 파일명을 설정합니다.
#SAVE_PATH = os.path.join(t_path, "inverse_net_final.pth")  # ✅ t_path로 수정
#torch.save(inverse_net.state_dict(), SAVE_PATH)
#print(f"\nModel saved successfully to {SAVE_PATH}")
# ==========================================================


[DEBUG] Epoch 1, Batch 1


KeyboardInterrupt: 

# **OPTUNA**

In [ ]:
# ────────────────────────────────────────────────────────────────────────────
# (C) InverseNet 정의 (MLP)
# ────────────────────────────────────────────────────────────────────────────
num_layers = len(material_sequence)  # 3
class InverseNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(401, 1198),
            nn.ReLU(),
            nn.Linear(1198, 1125),
            nn.ReLU(),
            nn.Linear(1125, 853),
            nn.ReLU(),
            nn.Linear(853, 1411),
            nn.ReLU(),
            nn.Linear(1411, 1320),
            nn.ReLU(),
            nn.Linear(1320, 1958),
            nn.ReLU(),
            nn.Linear(1958, 3)
        )

    def forward(self, x):
        # x must be shape (B, input_dim)
        return self.net(x)

# Re-initialize the model and optimizer after changing the class definition
inverse_net = InverseNet(input_dim=NUM_WAVELENGTHS, output_dim=num_layers).to(device)
inverse_net = inverse_net.to(torch.float64)
optimizer = optim.Adam(inverse_net.parameters(), lr=0.00018272261776066238)

# ────────────────────────────────────────────────────────────────────────────
# (D) 학습 루프: bi-TMM 방식
# ────────────────────────────────────────────────────────────────────────────
num_epochs = 600
save_interval = 600
output_dir = "C:/Users/PC/Desktop/Deep/"
os.makedirs(output_dir, exist_ok=True)
ALPHA = 1.0616785666328346
BETA = 18
for epoch in range(1, num_epochs + 1):
    # ---- train ----
    inverse_net.train()
    epoch_loss = 0.0
    for batch in train_loader:
        T_target = batch['T_target'].to(device)
        d_norm_true = batch['d_norm'].to(device)

        optimizer.zero_grad()
        x = T_target.squeeze(1)
        d_norm_pred = inverse_net(x)

        d_nm_pred = destandardize_thickness(d_norm_pred)
        T_pred_batch = model(d_nm_pred)  # ⬅️ 수정됨: no torch.no_grad()
        T_target_destd = destandardize_spectrum(x)

        loss_spectrum = F.mse_loss(T_pred_batch, T_target_destd)
        loss_thickness = F.mse_loss(d_norm_pred, d_norm_true.squeeze(1))
        loss = loss_spectrum + ALPHA * loss_thickness if epoch <= BETA else loss_spectrum

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # ---- validation ----
    inverse_net.eval()
    val_loss_total = 0.0
    with torch.no_grad():
        for val_batch in val_loader:
            T_val = val_batch['T_target'].to(device)
            d_norm_val = val_batch['d_norm'].to(device)

            x_val = T_val.squeeze(1).to(torch.float64)
            d_pred_val = inverse_net(x_val)
            d_nm_val_pred = destandardize_thickness(d_pred_val)
            T_val_pred = model(d_nm_val_pred)
            T_val_destd = destandardize_spectrum(x_val)

            loss_spectrum_v = F.mse_loss(T_val_pred, T_val_destd)
            loss_thickness_v = F.mse_loss(d_pred_val, d_norm_val.squeeze(1))
            val_loss = (loss_spectrum_v + ALPHA * loss_thickness_v) if epoch <= BETA else loss_spectrum_v
            val_loss_total += val_loss.item()

    avg_train_loss = epoch_loss / len(train_loader)
    avg_val_loss = val_loss_total / len(val_loader)
    if epoch % 1 == 0 or epoch == 1:
        print(f"Epoch[{epoch}/{num_epochs}]  train: {avg_train_loss:.6f}  val: {avg_val_loss:.6f}")

Epoch[1/600]  train: 0.135368  val: 0.118797
Epoch[2/600]  train: 0.100794  val: 0.097688
Epoch[3/600]  train: 0.091826  val: 0.085317
Epoch[4/600]  train: 0.081808  val: 0.077263
Epoch[5/600]  train: 0.073841  val: 0.072828
Epoch[6/600]  train: 0.066115  val: 0.061843
Epoch[7/600]  train: 0.059717  val: 0.054563
Epoch[8/600]  train: 0.057066  val: 0.058477
Epoch[9/600]  train: 0.054412  val: 0.053253
Epoch[10/600]  train: 0.050232  val: 0.047905
Epoch[11/600]  train: 0.049475  val: 0.046976
Epoch[12/600]  train: 0.047790  val: 0.040881
Epoch[13/600]  train: 0.046604  val: 0.042200
Epoch[14/600]  train: 0.045473  val: 0.043145
Epoch[15/600]  train: 0.041094  val: 0.033753
Epoch[16/600]  train: 0.040722  val: 0.035785
Epoch[17/600]  train: 0.037986  val: 0.035878
Epoch[18/600]  train: 0.034197  val: 0.044687
Epoch[19/600]  train: 0.008417  val: 0.004775
Epoch[20/600]  train: 0.005179  val: 0.004686
Epoch[21/600]  train: 0.004736  val: 0.003428
Epoch[22/600]  train: 0.004353  val: 0.0031

In [ ]:
DRIVE_PATH = "C:/Users/PC/Desktop/Deep/Materials_380"
DATA_DIR = DRIVE_PATH
# ==========================================================
#      [2. (참고) 나중에 저장된 모델 불러오기]
# ==========================================================
# 이 부분은 나중에 모델을 사용할 때 참고하시면 됩니다.

# 1. 먼저, 저장했을 때와 '똑같은 구조'의 모델 인스턴스를 생성합니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_to_load = InverseNet(input_dim=NUM_WAVELENGTHS, output_dim=num_layers).to(device)
model_to_load = model_to_load.to(torch.float64) # 저장 시 사용한 dtype과 동일하게 설정

# 2. 저장된 파라미터 파일(.pth)을 불러와 모델에 덮어씌웁니다.
LOAD_PATH = SAVE_PATH # 위에서 저장한 경로
model_to_load.load_state_dict(torch.load(LOAD_PATH))

# 3. 모델을 '평가 모드'로 전환합니다. (매우 중요!)
#    BatchNorm, Dropout 등의 레이어가 있을 경우, 추론 시에는 다르게 동작해야 하기 때문입니다.
model_to_load.eval()

print(f"\nModel loaded successfully from {LOAD_PATH}")

# 이제 model_to_load를 새로운 데이터 예측에 사용할 수 있습니다.
# 예시:
# with torch.no_grad(): # 예측 시에는 그래디언트 계산이 필요 없습니다.
#     some_new_spectrum = torch.rand(1, NUM_WAVELENGTHS, dtype=torch.float64).to(device)
#     predicted_thickness = model_to_load(some_new_spectrum)
#     print("\nPrediction with loaded model:", predicted_thickness)
# ==========================================================

In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os

# =================================================================================
# 이 상단에 (A)와 (B) 섹션의 코드를 그대로 유지합니다.
# (단, DataLoader 정의는 아래 objective 함수 안으로 이동했으므로 여기서는 제외)
# (drive.mount, material_files, material_data, WAVELENGTHS, TMMNetwork,
#  Dataset 클래스 정의, 표준화/역표준화 함수 등 모든 사전 준비 코드가 여기에 있어야 합니다.)
# =================================================================================


def objective(trial):
    """Optuna가 하이퍼파라미터 한 세트를 시험(trial)하는 함수"""
    seed_everything(42)

    # --- 1. 탐색할 하이퍼파라미터 정의 ---
    # 각 trial 마다 Optuna가 이 범위 안에서 새로운 값을 제안합니다.
    lr = trial.suggest_float("lr", 1e-4, 5e-4, log=True)
    alpha = trial.suggest_float("alpha", 0.1, 1.2, log=True)
    n_layers = trial.suggest_int("n_layers", 4, 4)  # 고정값 4로 제한
    n_units = trial.suggest_categorical("n_units", [768])  # 고정값 768
    batch_size = trial.suggest_categorical("batch_size", [32, 64])
    beta = trial.suggest_int("beta", 18, 23)
    num_epochs = trial.suggest_int("num_epochs", 180, 200)

    # --- 2. 제안된 하이퍼파라미터로 동적 컴포넌트 생성 ---

    # DataLoader: trial에서 제안된 batch_size를 사용해야 하므로 함수 내에서 생성
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    # InverseNet 정의: trial에서 제안된 n_layers, n_units를 사용
    class InverseNet(nn.Module):
        def __init__(self, input_dim, output_dim, num_hidden_layers, units):
            super().__init__()
            layers = []

            # Input Layer
            layers.append(nn.Linear(input_dim, units))

            layers.append(nn.ReLU())

            # Hidden Layers
            for _ in range(num_hidden_layers):
                layers.append(nn.Linear(units, units))

                layers.append(nn.ReLU())

            # Output Layer
            layers.append(nn.Linear(units, output_dim))

            self.net = nn.Sequential(*layers)

        def forward(self, x):
            return self.net(x)

    # 모델 및 옵티마이저 생성: trial에서 제안된 값들로 인스턴스화
    num_layers_output = len(material_sequence)
    inverse_net = InverseNet(NUM_WAVELENGTHS, num_layers_output, n_layers, n_units).to(device)
    inverse_net = inverse_net.to(torch.float64)
    optimizer = optim.Adam(inverse_net.parameters(), lr=lr)

    # --- 3. 기억하고 있는 학습 루프 실행 ---


    for epoch in range(num_epochs):
        inverse_net.train()
        total_loss = 0.0
        for batch_idx, batch in enumerate(dataloader):
            T_target = batch['T_target'].to(device)
            d_norm_true = batch['d_norm'].to(device)

            optimizer.zero_grad()

            squeezed_T_target = T_target.squeeze(1)
            d_norm_pred = inverse_net(squeezed_T_target)

            d_nm_pred = destandardize_thickness(d_norm_pred)
            T_pred_batch = model(d_nm_pred)

            T_target_destd = destandardize_spectrum(squeezed_T_target)

            loss_spectrum = F.mse_loss(T_pred_batch, T_target_destd)
            loss_thickness = F.mse_loss(d_norm_pred, d_norm_true.squeeze(1))

            # 2단계 학습 전략 (기억하고 있는 코드 로직)
            if epoch < beta:
                loss = loss_spectrum + alpha * loss_thickness # trial에서 제안된 alpha 사용
            else:
                loss = loss_spectrum

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

    # 최종 epoch의 평균 Loss를 이 trial의 성능 지표로 반환
    final_avg_loss = total_loss / len(dataloader)

    # Pruning을 위해 중간 결과 보고
    trial.report(final_avg_loss, epoch)
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return final_avg_loss

# --------------------------------------------------------------------------
# Optuna Study 실행
# --------------------------------------------------------------------------
if __name__ == "__main__":
    # 'minimize' 방향으로 objective 함수(loss)를 최적화하는 연구(study) 생성
    study = optuna.create_study(
        direction="minimize",
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=3),
        sampler=optuna.samplers.TPESampler(seed=42)
    )

    study.optimize(objective, n_trials=100, n_jobs=1)  # CPU-only 병렬이면 n_jobs=4 가능

    # --- 최적 결과 확인 ---
    print("\n\n========================================================")
    print("                      탐색 완료!                      ")
    print("========================================================")
    print("최적의 Trial 번호:", study.best_trial.number)
    print("최적의 Loss (Value):", study.best_trial.value)
    print("\n최적의 하이퍼파라미터 (Best Params):")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")

[I 2025-06-15 17:43:24,246] A new study created in memory with name: no-name-178cfa68-0202-4323-ab29-12fd772c5cf5
[I 2025-06-15 18:10:12,062] Trial 0 finished with value: 0.0007120522152449268 and parameters: {'lr': 0.00018272261776066238, 'alpha': 1.0616785666328346, 'n_layers': 4, 'n_units': 768, 'batch_size': 32, 'beta': 18, 'num_epochs': 183}. Best is trial 0 with value: 0.0007120522152449268.
[I 2025-06-15 18:36:48,236] Trial 1 finished with value: 0.0031391777486834316 and parameters: {'lr': 0.00010979908036596662, 'alpha': 0.8605201659336437, 'n_layers': 4, 'n_units': 768, 'batch_size': 64, 'beta': 18, 'num_epochs': 200}. Best is trial 0 with value: 0.0007120522152449268.
[W 2025-06-15 18:44:45,774] Trial 2 failed with parameters: {'lr': 0.0003818145165896873, 'alpha': 0.16949324171244332, 'n_layers': 4, 'n_units': 768, 'batch_size': 64, 'beta': 19, 'num_epochs': 191} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\PC\Desk

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
num_layers = len(material_sequence)  # 3
class InverseNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.ReLU(),
            nn.Linear(768, output_dim),
            # no nonlinearity here
        )

    def forward(self, x):
        # x must be shape (B, input_dim)
        return self.net(x)

# ---------------------------
# 1. 저장된 모델 로드
# ---------------------------
inverse_net = InverseNet(input_dim=NUM_WAVELENGTHS, output_dim=len(material_sequence)).to(device)
inverse_net = inverse_net.to(torch.float64)

# 모델 파라미터 로드
ckpt_path = os.path.join("C:/Users/PC/Desktop/Deep/", "inverse_net_final.pth")
inverse_net.load_state_dict(torch.load(ckpt_path, map_location=device))
inverse_net.eval()  # 평가모드
print(f"모델 로드 완료: {ckpt_path}")

# ---------------------------
# 2. 첫 배치만 inference
# ---------------------------
for batch_idx, batch in enumerate(dataloader):
    d_norm_true = batch['d_norm'].to(device)     # (B, 3)
    T_target = batch['T_target'].to(device)      # (B, 401)

    # (1) Target 출력
    d_true_nm = destandardize_thickness(d_norm_true)
    print("=== d_true (nm) ===")
    print(d_true_nm[0].cpu().numpy())
    print("=== T_target 스펙트럼 ===")
    print("min, max, mean = ",
          T_target[0].min().item(), T_target[0].max().item(), T_target[0].mean().item())

    # (2) 두께 예측
    with torch.no_grad():
        d_pred_norm = inverse_net(T_target)  # (B, 3)
        d_pred_nm = destandardize_thickness(d_pred_norm)

    print("\n=== d_pred_norm (표준화된 두께) ===")
    print(d_pred_norm[0].cpu().numpy())
    print("→ d_pred_nm =", d_pred_nm[0].cpu().numpy())

    # (3) TMM forward (1개씩)
    T_pred_list = []
    for i in range(T_target.shape[0]):
        thickness_nm = d_pred_nm[i]  # (3,)
        T_pred = model(thickness_nm.to(torch.float64))  # (401,)
        T_pred_list.append(T_pred)

        if batch_idx == 0 and i == 0:
            print("→ TMM 입력 두께 =", thickness_nm.cpu().numpy())
            print("TMM 결과 시각화:")

            plt.plot(WAVELENGTHS, T_pred.detach().cpu().numpy(), label='T_pred')
            plt.plot(WAVELENGTHS, T_target[i].detach().cpu().numpy(), label='T_target', linestyle='dashed')
            plt.xlim(380, 780)
            plt.ylim(0, 1)
            plt.xlabel("Wavelength (nm)")
            plt.ylabel("Transmittance")
            plt.grid(True)
            plt.legend()
            plt.show()

    # (4) MSE 계산
    T_pred_batch = torch.stack(T_pred_list, dim=0)  # (B, 401)
    mse_loss = F.mse_loss(T_pred_batch, T_target)
    print(f"\n첫 배치 전체 MSE loss = {mse_loss.item():.6f}")
    break  # 첫 배치만 체크하고 종료


C:\Users\PC\AppData\Local\Temp\ipykernel_29672\3715825692.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  inverse_net.load_state_dict(torch.load(ckpt_path, map_location

모델 로드 완료: C:/Users/PC/Desktop/Deep/inverse_net_final.pth
=== d_true (nm) ===
[[  10. 1300.   15.]]
=== T_target 스펙트럼 ===
min, max, mean =  -0.18730372650155494 3.612421506591234 0.8806223545470996

=== d_pred_norm (표준화된 두께) ===
[[-1.32444453  1.22742783 -0.80030322]]
→ d_pred_nm = [[  10.63476292 1305.11102745   14.34100166]]
→ TMM 입력 두께 = [[  10.63476292 1305.11102745   14.34100166]]
TMM 결과 시각화:


ValueError: x and y must have same first dimension, but have shapes (401,) and (1, 401)